# Neurons tutorial

<img src="../Docs/Media/m1th_2.png" width="600" height="350">



brainrender let's you visualize neuron morphology in the context of the Allen Mouse brain atlas. 
Currently brainrender can visualize any morpholy data presented as .swc file, but only supports direct interaction with the MouseLight API. In the future, .swc data from the Allen API will also be available. 

To get morphology data from Janelia's MouseLight, you have to options (see [the userguid](../Docs/UserGuide.md) for more details.: 
* Download them manually from the Neurons Browser online
* Download them using brainrender.

In this tutorial we will focus on using brainrender to fetch neurons data and render it, if you've downloaded the data alread skip the fetching and get started renering!

### Set up

In [ ]:
# We begin by adding the current path to sys.path to make sure that the imports work correctly
import sys
sys.path.append('../')
import os

# Set up VTKPLOTTER to work in Jupyter notebooks
from vtkplotter import *
embedWindow(backend=False) 

# Import variables
from brainrender import * # <- these can be changed to personalize the look of your renders

# Import brainrender classes and useful functions
from brainrender.scene import Scene
from brainrender.Utils.parsers.mouselight import NeuronsParser
from brainrender.Utils.MouseLightAPI.mouselight_api import MouseLightAPI
from brainrender.Utils.MouseLightAPI.mouselight_info import mouselight_api_info, mouselight_fetch_neurons_metadata
from brainrender.Utils.data_io import listdir

# Before populating the scene, we need to change the current working directory to the parent folder, 
# then we are ready to start!
os.chdir(os.path.normpath(os.path.join(os.getcwd(), os.pardir)))

## Mouse Light API
This shows you how to download morphology data using brainrender, if you have the data alread, skip ahead to the next section.

In [ ]:
# To see how many neurons are available in the Mouse Light database:
print(mouselight_api_info())

In [ ]:
# To download neurons, first we need to get metdata about the neurons in the database
neurons_metadata = mouselight_fetch_neurons_metadata()

# If instead we want to only get neurons whose soma is in a region of interest:
print("\nLet's get only the neurons in the secondary motor cortex.")
neurons_metadata = mouselight_fetch_neurons_metadata(filterby='soma', filter_regions=['MOs'])

# Then we can download the files and save them as a .json file
# First create an instance of the mouselight API class
ml_api = MouseLightAPI() 

# Then download neurons
neurons_files =  ml_api.download_neurons(neurons_metadata[:2]) # just saving the first couple neurons to speed things up

Note that, since the neurons morphologies are saved as a .json file for each neuron, you'll have to download the data only once. If you are rendering a neuron for the second time, brainrender will simply fetch the saved data. 

### Render neurons

In [ ]:
# if you didn't download the files above, you can get one from the examples files
neurons_files = "Examples/example_files/one_neuron.json"

In [ ]:
# Create the actors for the neurons to render
# create an instance of the neuron parser class, passing the options to specify how to render the neurons
parser = NeuronsParser(scene=tutorial_scene, 
                         color_neurites=True, axon_color="antiquewhite", 
                         soma_color="darkgoldenrod", dendrites_color="firebrick")

# Then simply render neurons
neurons, regions = parser.render_neurons(neurons_files)

# You can also specify the rendering options by passing the  keyword arguments to .render_neurons

Parsing neurons is a slow business. For this reason brainrender caches the results of the parsing, making subsequent renderings of the same neuron very fast!

In [ ]:
# Show neurons and ZI in the same scene:
tutorial_scene = Scene()
tutorial_scene.add_neurons(neurons) # ! don't forget to add the neurons actors to the scene
tutorial_scene.add_brain_regions(['ZI'], colors='white', alpha=0.5) # add the ZonaIncerta to our scene
tutorial_scene.render() 

You can also create the neurons models and add them to the scene in one step

In [ ]:
tutorial_scene = Scene()
tutorial_scene.add_neurons(neurons_files, soma_color='red', color_neurites=False)# this will take care of rendering and adding to scene
tutorial_scene.add_brain_regions(['ZI'], colors='white', alpha=0.5) # add the ZonaIncerta to our scene
tutorial_scene.render()

But rendering multiple neurons may take a couple minutes, so you can also save the actors as a variable and use 
them repeatedly

In [ ]:
tutorial_scene = Scene()

neurons_files = "Examples/example_files/axons_in_CA1.json"
# To color soma, dendrites and axon with different colors, set color_neurites as True and 
# specify the colors for the different parts of the neurons
neurons, regions = parser.render_neurons(neurons_files, scene=tutorial_scene, 
                         color_neurites=True, axon_color="antiquewhite", 
                         soma_color="darkgoldenrod", dendrites_color="firebrick")

In [ ]:
tutorial_scene.add_neurons(neurons) # you can pass keyword arguments to this function to edit the look of the neurons
# it takes the same arguments as "render_neurons"
tutorial_scene.add_brain_regions(['CA1'], colors='green', alpha=0.5) # add the ZonaIncerta to our scene
tutorial_scene.render() 

If you have a scene with some neurons already in it. You can change their appearence with `.edit_neurons`, withouth having to render them again

### Other ways to change the aspect of neurons

In [ ]:
# You can also make it so that each neuron has a different random color.
tutorial_scene = Scene()

neurons_files = "Examples/example_files/neurons_in_Striatum.json"
first, _ = parser.render_neurons(neurons_files, scene=tutorial_scene, color_neurites=False, random_color=True)
neurons_files = "Examples/example_files/one_neuron.json"
second, _ = parser.render_neurons(neurons_files, scene=tutorial_scene, color_neurites=False, random_color=True)


tutorial_scene.add_neurons(first)
tutorial_scene.add_neurons(second)

tutorial_scene.render() 

You can pass a string (e.g. `hot`) as value for `random_color` to have the neurons have a random color drawn from a colormap. 

In [ ]:
# Finally, you can decide how big to make the neurites, or not to show them at all
tutorial_scene = Scene()
tutorial_scene.add_neurons(neurons_file, soma_color='red', render_neurites=False)
tutorial_scene.render()

In [ ]:
tutorial_scene = Scene()
tutorial_scene.add_neurons(neurons_file, soma_color='red', neurites_radius=50)
tutorial_scene.render()

### Other options
You can also have the brain regions that the neurons go through rendered. 

In [ ]:
# Create a scene, add neurons and show the regions that the axons go through. 
neurons_file = "Examples/example_files/neurons_in_Striatum.json"

scene = Scene()
scene.add_neurons(neuron_file, color_neurites=False, random_color="jet", display_axon_regions=True)
scene.render()

There are more options that you can use to change the look of the rendered neurons:
* force_to_hemisphere: makes sure that the soma of all neurons is in the hemisphere of your choice. 
* color_by_region: gives the neurons the standard allen mouse atlas color for the region their soma is in
* mirror: crates a mirror version of each neuron in both hemispheres. 

Don't forget that you can also change the appearance of rendered neurons by using the 'edit_neurons' function of the Scene class. 

Don't forget to check the other examples to lear more about how to use brainrender to make amazing 3D renderings!
Also, you can find a list of variables you can play around with in brainrender.variables.py
Playing around with these variables will allow you to make the rendering look exactly how you want them to be. 